# Machine Learning in A/B Testing & Causal Inference

Traditional A/B testing relies on t-tests and chi-square tests, but machine learning (ML), particularly regression models, can:
- Handle multiple covariates to reduce variance.
- Adjust for confounders in observational studies.
- Provide better interpretability when sample sizes are small.<br>

This would cover linear regression (for continuous outcomes) and logistic regression (for binary outcomes) as equivalents to classical statistical tests.

| Outcome Type                  | Traditional Test       | ML Equivalent        |
|:------------------------------|:----------------------|:---------------------|
| Continuous (Revenue, Time)     | t-test                | Linear Regression   |
| Binary (Conversion, Clicks)    | Chi-Square / Z-test   | Logistic Regression |


## Linear Regression for A/B Testing (Continuous Outcomes)

The classic t-test checks if two groups have significantly different means. Instead, we can fit a linear regression model:

$$
Y_i = \beta_0 + \beta_1 \cdot \text{Treatment}_i + \epsilon_i
$$

Where:  

- $Y_i$ = Outcome (e.g., revenue per user)  
- Treatment is a binary variable ($0 =$ Control, $1 =$ Treatment)  
- $\beta_1$ tells us the effect of treatment (like the t-test statistic)
- If $\beta_1$ is significant (p < 0.05), treatment had an effect!


In [13]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Simulate data
np.random.seed(42)
n = 1000
df = pd.DataFrame({
    'Treatment': np.random.choice([0, 1], size=n),  # 0 = Control, 1 = Treatment
})

# Treatment adds between 0 to 4 revenue units
df['Revenue'] = np.random.normal(50, 10, size=n) + df['Treatment'] * np.random.choice([0, 4], size=n) 

# Fit linear regression
X = sm.add_constant(df['Treatment'])
model = sm.OLS(df['Revenue'], X).fit()

# Print results
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:                Revenue   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                  0.009
Method:                 Least Squares   F-statistic:                     10.06
Date:                Tue, 25 Feb 2025   Prob (F-statistic):            0.00156
Time:                        15:35:26   Log-Likelihood:                -3732.8
No. Observations:                1000   AIC:                             7470.
Df Residuals:                     998   BIC:                             7479.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         50.4618      0.457    110.338      0.0

## Logistic Regression for A/B Testing (Binary Outcomes)

When the outcome is binary (e.g., conversion: 1 = purchased, 0 = not purchased), logistic regression can replace a proportion test (Z-test or chi-square test):

$$
P(Y = 1) = \frac{1}{1 + e^{-(\beta_0 + \beta_1 \cdot \text{Treatment})}}
$$

Where:
- $Y$ is the conversion (1 or 0)
- $\beta_1$ tells us how treatment affects the odds of conversion
-  If $\beta_1$ is significant (p < 0.05), the treatment significantly affects conversion rates!  

In [14]:
import statsmodels.formula.api as smf

# Simulate data
np.random.seed(42)
n = 1000
df = pd.DataFrame({
    'Treatment': np.random.choice([0, 1], size=n),  # 0 = Control, 1 = Treatment
})

# Simulate conversion data
df['Conversion'] = np.random.binomial(n=1, p = 0.1 + 0.05 * df['Treatment'])  # Treatment boosts conversion by 5%

# Fit logistic regression
model = smf.logit("Conversion ~ Treatment", data=df).fit()

# Print results
print(model.summary())


Optimization terminated successfully.
         Current function value: 0.374155
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:             Conversion   No. Observations:                 1000
Model:                          Logit   Df Residuals:                      998
Method:                           MLE   Df Model:                            1
Date:                Tue, 25 Feb 2025   Pseudo R-squ.:                 0.01203
Time:                        15:38:22   Log-Likelihood:                -374.15
converged:                       True   LL-Null:                       -378.71
Covariance Type:            nonrobust   LLR p-value:                  0.002537
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -2.2672      0.155    -14.637      0.000      -2.571      -1.964
Treatment      0.5854      0.

## Causal Inference with Regression

Regression can also control for confounders in observational data:

$$
Y = \beta_0 + \beta_1 \cdot \text{Treatment} + \beta_2 \cdot X_1 + \beta_3 \cdot X_2 + \dots + \epsilon
$$

Where: 
- $X_1, X_2$ = Confounding variables (e.g., age, region, prior behavior).  
- $\beta_1$ tells us the true causal effect of treatment after adjusting for other factors.  


In [26]:
# Simulate data
np.random.seed(42)
n = 1000
df = pd.DataFrame({
    'Treatment': np.random.choice([0, 1], size=n),  # 0 = Control, 1 = Treatment
})

# Treatment adds between 0 to 4 revenue units
df['Revenue'] = np.random.normal(50, 10, size=n) + df['Treatment'] * np.random.choice([0, 4], size=n) 

df['Age'] = np.random.randint(18, 65, size=n)
df['Device'] = np.random.choice(['Desktop', 'Mobile'], size=n)

# Encode categorical variable
df = pd.get_dummies(df, columns=['Device'], drop_first=True).astype(int)

# Fit multiple regression
X = sm.add_constant(df[['Treatment', 'Age', 'Device_Mobile']])
model = sm.OLS(df['Revenue'], X).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                Revenue   R-squared:                       0.015
Model:                            OLS   Adj. R-squared:                  0.012
Method:                 Least Squares   F-statistic:                     4.997
Date:                Tue, 25 Feb 2025   Prob (F-statistic):            0.00192
Time:                        15:57:26   Log-Likelihood:                -3728.7
No. Observations:                1000   AIC:                             7465.
Df Residuals:                     996   BIC:                             7485.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const            48.4249      1.146     42.270